### Estimator - Basic Regression Model

In [9]:
import tensorflow as tf
import numpy as np

In [11]:
x_train = np.array([1.0,2.0,3.0,4.0])
y_train = np.array([-1.0,-2.0,-3.0,-4.0])


#Testing Inputs and Outputs
x_eval = np.array([5.0, 10.0, 15.0,20.0])
y_eval = np.array([-5.0, -10.0, -15.0,-20.0])

# Prediction Inputs
x_predict = np.array([50.0, 100.0])


# Inputs into our model. For each feature/factor we are inputting, we need another feature column
# Think about these as input nodes
feature_column = tf.feature_column.numeric_column(key='x', shape=[1])
feature_columns = [feature_column]

# Type of estimator we are going to use (in this case Linear Regression)
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# Input function for training the model using training data
input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_train},
                                             y=y_train,
                                             batch_size=4,
                                             num_epochs=None,
                                             shuffle=True)

# Input function for evaluating the training phase of the model using training data
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_train},
                                             y=y_train,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

# Input function for testing the model using testing data
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_eval},
                                             y=y_eval,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

# Input function for predicting using some values
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_predict},
                                             
                                             num_epochs=1,
                                             shuffle=False)

# Run the training phase with 1000 epochs and the training fucntion
estimator.train(input_fn=input_fn, steps=1000)
# Evaluate the training phase
print(estimator.evaluate(input_fn=train_input_fn))

# Evaluate the testing phase (final Pperformance)
print(estimator.evaluate(input_fn=eval_input_fn))

# Predict using some inputs and our trained model
print(list(estimator.predict(input_fn=predict_input_fn)))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Sunny\\AppData\\Local\\Temp\\tmpnmqrwnst', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002C1B8355DA0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkp

### Using Custom Estimator Function
To replace the above Regressor Model
```
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)
```
using our custom Estimator i.e. `model_fn`

In [14]:
import tensorflow as tf
import numpy as np

x_train = np.array([1.0,2.0,3.0,4.0])
y_train = np.array([-1.0,-2.0,-3.0,-4.0])


#Testing Inputs and Outputs
x_eval = np.array([5.0, 10.0, 15.0,20.0])
y_eval = np.array([-5.0, -10.0, -15.0,-20.0])

# Prediction Inputs
x_predict = np.array([50.0, 100.0])


In [19]:
# y = Wx+b
def model_fn(features, labels, mode):
    #if mode == 'infer':
        #labels = np.array([0,0])
    W = tf.get_variable(name='W', shape=[1], dtype=tf.float64)
    b = tf.get_variable(name='b', shape=[1], dtype=tf.float64)
    y = W * features['x'] + b
    
    loss = tf.reduce_sum(input_tensor=tf.square(x=(y-labels)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    global_step = tf.train.get_global_step()
    
    train_step = tf.group(optimizer.minimize(loss=loss),tf.assign_add(global_step, 1))
    return tf.estimator.EstimatorSpec(
        mode = mode,
        predictions = y,
        loss = loss,
        train_op = train_step
    )

In [21]:
# Inputs into our model. For each feature/factor we are inputting, we need another feature column
# Think about these as input nodes
feature_column = tf.feature_column.numeric_column(key='x', shape=[1])
feature_columns = [feature_column]

# Type of estimator we are going to use (in this case Linear Regression)
estimator = tf.estimator.Estimator(model_fn=model_fn)

# Input function for training the model using training data
input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_train},
                                             y=y_train,
                                             batch_size=4,
                                             num_epochs=None,
                                             shuffle=True)

# Input function for evaluating the training phase of the model using training data
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_train},
                                             y=y_train,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

# Input function for testing the model using testing data
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_eval},
                                             y=y_eval,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

# Input function for predicting using some values
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': x_predict},
                                             
                                             num_epochs=1,
                                             shuffle=False)

# Run the training phase with 1000 epochs and the training fucntion
estimator.train(input_fn=input_fn, steps=1000)
# Evaluate the training phase
print(estimator.evaluate(input_fn=train_input_fn))

# Evaluate the testing phase (final Pperformance)
print(estimator.evaluate(input_fn=eval_input_fn))

# Predict using some inputs and our trained model
print(list(estimator.predict(input_fn=predict_input_fn)))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Sunny\\AppData\\Local\\Temp\\tmpfaik4pw5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002C1B82DAB70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkp